In [ ]:
%%sh
tar tvfz model.tar.gz

In [ ]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

prefix = 'byo-tf'

In [ ]:
model_path = sess.upload_data(path='model.tar.gz', key_prefix=prefix)
print(model_path)

In [ ]:
from sagemaker.tensorflow.serving import Model

tf_model = Model(model_path,
           role,
           framework_version='2.1.0')


In [ ]:
type(tf_model)

In [ ]:
import time

tf_endpoint_name = 'keras-tf-fmnist-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

tf_predictor = tf_model.deploy(
                 initial_instance_count=1, 
                 instance_type='ml.m5.large',
                 endpoint_name=tf_endpoint_name)

In [ ]:
type(tf_predictor)

In [ ]:
import os
import numpy as np
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

os.makedirs("./data", exist_ok = True)
np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_val.shape[0] - 1), num_samples)
images = x_val[indices]/255
labels = y_val[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')

payload = images.reshape(num_samples, 28, 28, 1)

In [ ]:
# Default format for inference is JSON

from sagemaker.predictor import json_serializer
tf_predictor.content_type = 'application/json'
tf_predictor.serializer =  json_serializer

In [ ]:
response = tf_predictor.predict(payload)

prediction = np.array(response['predictions'])
predicted_label = prediction.argmax(axis=1)
print('Predicted labels are: {}'.format(predicted_label))

## Build another predictor

In [ ]:
from sagemaker.tensorflow.serving import Predictor

another_predictor = Predictor(tf_endpoint_name)

In [ ]:
from sagemaker.predictor import json_serializer

another_predictor.content_type = 'application/json'
another_predictor.serializer =  json_serializer

In [ ]:
response = another_predictor.predict(payload)

prediction = np.array(response['predictions'])
predicted_label = prediction.argmax(axis=1)
print('Predicted labels are: {}'.format(predicted_label))

## Update the endpoint

In [ ]:
tf_predictor = tf_model.deploy(initial_instance_count=2,
                               instance_type='ml.m5.large',
                               endpoint_name=tf_endpoint_name,
                               update_endpoint=True)

In [ ]:
tf_predictor.delete_endpoint()